In [63]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.pipeline import Pipeline
from sklearn import set_config
from matplotlib import style
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler 
from sklearn.cluster import KMeans

In [64]:
import models
hankel = models.Hankel()
rank = models.Rank()
cluster = models.Cluster()
meepc = models.MEEPC() 

In [65]:
df = pd.read_csv('~/data/training_data.csv')

In [66]:
df

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
1,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
2,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
3,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
4,167.895,0.619473,11.6179,504.145,0.293009,0.001080,62.1472,1.0,1.0,1.0,...,0.0,8279.1,14.00090,264.802,0.620240,0.672394,67.1004,60.1557,0.11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971995,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971996,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971997,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971998,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0


In [67]:
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]

In [68]:
df_normal

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
1,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
2,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
3,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
4,167.895,0.619473,11.6179,504.145,0.293009,0.001080,62.1472,1.0,1.0,1.0,...,0.0,8279.1,14.00090,264.802,0.620240,0.672394,67.1004,60.1557,0.11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971938,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971939,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971940,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971941,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0


In [69]:
df_attack

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
941103,167.005,0.613474,11.9575,467.477,0.293415,0.000870,44.9750,1.0,1.0,1.0,...,0.0,8279.1,7.33387,633.337,0.491560,0.001369,64.4316,63.2852,0.69,-1.0
941104,167.005,0.613474,11.9575,467.477,0.293415,0.000870,44.9750,1.0,1.0,1.0,...,0.0,8279.1,7.33387,633.337,0.491560,0.001369,64.4316,63.2852,0.69,-1.0
941105,167.005,0.613474,11.9575,467.477,0.293415,0.000870,44.9750,1.0,1.0,1.0,...,0.0,8279.1,7.33387,633.337,0.491560,0.001369,64.4316,63.2852,0.69,-1.0
941106,166.965,0.619474,11.9573,467.522,0.293185,0.001254,45.1677,1.0,1.0,1.0,...,0.0,8279.1,7.26840,638.484,0.491449,0.001392,64.3614,63.5099,0.69,-1.0
941107,166.965,0.619474,11.9573,467.522,0.293185,0.001254,45.1677,1.0,1.0,1.0,...,0.0,8279.1,7.26840,638.484,0.491449,0.001392,64.3614,63.5099,0.69,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971995,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971996,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971997,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971998,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0


In [70]:
sensorvalue=2

In [71]:
sensors = [3,9,25,43]
# sensors = [21,22,23,24,25,26,27,28,29,30]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [72]:
def calc_width_height(v):
    width = 2/np.sqrt(v) 
    return width
    
def draw_ellipse(i,j,centers,widths,X):
    plt.xlim(centers[i]-widths[i]/2,centers[i]+widths[i]/2)
    plt.ylim(centers[j]-widths[j]/2,centers[j]+widths[j]/2)
    ellipse = Ellipse(xy=(centers[i], centers[j]), width=widths[i], height=widths[j])
    ellipse.fill = False
    ellipse.set_ec('red')
    ax.add_patch(ellipse)
    plt.scatter(x = X[::,i], y = X[::,j])
    

In [73]:
X = df_normal.iloc[:,sensorvalue].values
X = hankel.fit(X,10,0.5)
X=X.T 
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
radius1=[]

for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V[:,:r])
    cluster_ = np.matmul(cluster_,V[:,:r])
    print(cluster_.shape)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    radius1.append(np.sqrt(var2))
    max_radius=math.sqrt(np.amax(var2))
    radiis.append(max_radius)
    
print(radiis)

4
r: 11
(1553, 11)
r: 9
(1335, 9)
r: 9
(166, 9)
r: 9
(156, 9)
[1.000000930168902, 1.0000007431025906, 0.999999537671207, 1.0001279549432047]


# attack training


In [74]:
X_attack = df_attack.iloc[:,sensorvalue].values
X_att = hankel.fit(X_attack,10,0.5)
X_att=X_att.T 
print(X_att.shape)
radius2=[]
for i in range(optimal_k):
    cluster_=np.matmul(X_att,clusters_V[i])
    print(cluster_.shape)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    radius2.append(np.sqrt(var2))
radius2

(27, 600)
(27, 11)
(27, 9)
(27, 9)
(27, 9)


[array([0.62879385, 0.64349282, 0.57903111, 0.51958982, 0.90194834,
        0.83165752, 1.00022752, 0.64578362, 0.61512102, 0.646868  ,
        0.68188236, 0.9265774 , 0.93936708, 0.77727591, 0.69110185,
        0.68493322, 0.67527925, 0.67591712, 0.70503958, 0.75048241,
        0.66814926, 0.99999034, 1.00000276, 0.79941599, 0.88688914,
        0.72965449, 0.30010689]),
 array([0.78864431, 0.76092992, 0.70479356, 0.6275699 , 0.99980461,
        1.00013336, 0.9103753 , 0.75886348, 0.74757269, 0.78774363,
        0.83220894, 0.99105142, 0.95323229, 0.83616777, 0.83851834,
        0.82994169, 0.82991771, 0.83261539, 0.81316466, 0.82968013,
        0.83005754, 0.94776667, 0.99999869, 0.8375196 , 0.88862009,
        0.73405945, 0.35000432]),
 array([0.65593327, 0.68827229, 0.71800324, 0.55668159, 0.70040088,
        0.6457194 , 0.67026643, 0.67187248, 0.63692923, 0.66736772,
        0.69465885, 0.99999949, 0.93517587, 0.73865988, 0.61687256,
        0.61063117, 0.60367828, 0.60903114, 0.60

In [75]:
# X_attack = df_attack.iloc[:,sensorvalue].values
# X_att = hankel.fit(X_attack,10,0.5)
# X_att=X_att.T 
# kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
# kmeans.fit(X_att)
# radiis2=[]
# print(X_att.shape)
# radius2=[]
# for i in range(optimal_k):
#     cluster_ = X_att[np.where(kmeans.labels_ == i)[0]]
#     r = rank.fit(cluster_)
#     # clusters_R.append(r)
#     print("r: "+str(r))
#     U,Sigma,VT = np.linalg.svd(cluster_)
#     V = VT.T
#     clusters_V.append(V)
#     cluster_ = np.matmul(cluster_,V[:,:r])
#     print(cluster_.shape)
#     # cluster_=tsne.fit_transform(cluster_)
#     weight,center = meepc.fit(cluster_)
#     weights.append(weight)
#     centers.append(center)
#     widths=calc_width_height(weight)
#     var1=np.square(cluster_-center)
#     var2=np.matmul(weight,var1.T)
#     radius2.append(np.sqrt(var2))
#     # print(var2.shape)
#     max_radius=math.sqrt(np.amax(var2))
#     radiis2.append(max_radius)
# print(radiis2)

In [103]:
def threshold(radius1,radius2,optimal_k):
    threshold=[]
    radius3=[]
    mapping=[]
    argsort_indices=[]
    
    #appending radiis of positives and negatives in radius3
    for i in range(optimal_k):
        radius3.append(np.concatenate((radius1[i],radius2[i])))
    
    #created a mapping +1 for normal radius and -1 for attack radius
    for i in range(optimal_k):
        mapping.append(np.concatenate((np.ones(radius1[i].shape[0]),np.negative(np.ones(radius2[i].shape[0])))))
    
    #for each cluster finding threshold
    for i in range(optimal_k):
        argsort_indices.append(np.argsort(radius3[i]))
    for cluster in range(optimal_k): 
        positives=[]
        negatives=[]
        labels=[]
        for idx in argsort_indices[cluster]:
            labels.append(mapping[cluster][idx])
        neg=np.cumsum((1-np.array(labels))/2)
        pos=np.cumsum((1+np.array(labels))/2)
        no_pos=np.sum(1+np.array(labels)/2)
        fn=no_pos-pos

        #fscore
        final_radius=(2*pos)/(2*pos+fn+neg)
        # final_radius=(2/(3+(neg/pos)))
        # print(final_radius)   
        idx=np.argmax(final_radius)     
        radiii=radius3[cluster][argsort_indices[cluster][idx]]
        threshold.append(radiii)

    return threshold

In [83]:
# start=0
# for idx in argsort_indices[cluster]:
#     if(start==0 and mapping[cluster][idx]==1):
#         positives.append(1)
#         start=1
#     elif(start==0 and mapping[cluster][idx]==-1):
#         positives.append(0)
#         start=1
#     elif(mapping[cluster][idx]==-1):
#         positives.append(positives[-1])
#     elif(mapping[cluster][idx]==1):
#         positives.append(positives[-1]+1)

# start=0
# for idx in argsort_indices[cluster]:
#     if(start==0 and mapping[cluster][idx]==1):
#         negatives.append(0)
#         start=1
#     elif(start==0 and mapping[cluster][idx]==-1):
#         negatives.append(1)
#         start=1
#     elif(mapping[cluster][idx]==-1):
#         negatives.append(negatives[-1]+1)
#     elif(mapping[cluster][idx]==1):
#         negatives.append(negatives[-1])
# final_list=np.array(positives)-np.array(negatives)
# print(argsort_indices[cluster][np.argmax(final_list)])
# thresh=radius3[cluster][argsort_indices[cluster][np.argmax(final_list)]]
# threshold.append(thresh)

In [104]:
radius_cluster=threshold(radius1,radius2,optimal_k)

In [105]:
radius_cluster

[1.000000930168902, 0.998157468152326, 0.9986001021045486, 1.0001279549432047]

# testing 

In [106]:
df_test=pd.read_csv('~/data/testdata.csv')
df_test

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
1,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
2,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
3,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
4,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248395,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248396,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248397,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248398,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0


In [107]:
y_actual=df_test.iloc[:,-1]
# df_test_normal=df_test[df_test['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_test=df_test.iloc[:,sensors]
df_test

,1_AIT_004_PV,1_MV_003_STATUS,2_FIC_401_CO,2_LS_101_AH
0,463.716,1.0,20.2507,0.0
1,463.716,1.0,20.2507,0.0
2,463.716,1.0,20.2507,0.0
3,463.716,1.0,20.2507,0.0
4,463.716,1.0,20.2507,0.0
...,...,...,...,...
248395,430.680,1.0,19.2170,0.0
248396,430.680,1.0,19.2170,0.0
248397,430.680,1.0,19.2170,0.0
248398,430.680,1.0,19.2170,0.0


In [108]:
labels=hankel.fit(np.array(y_actual),10,0.5)
labels

array([[ 1.,  1.,  1., ..., -1.,  1.,  1.],
       [ 1.,  1.,  1., ..., -1.,  1.,  1.],
       [ 1.,  1.,  1., ..., -1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [109]:
labels.shape

(600, 826)

In [110]:
isatt=np.any(labels<0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823])

In [111]:
X_test=df_test.iloc[:,sensorvalue].values
Xtest=hankel.fit(X_test,10,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
print(Xtest.shape)
distances=[]
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) #size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    distance=np.sqrt(var2)
    distances.append(distance)

(826, 600)


In [112]:
distances[0].shape #size (274,)

(826,)

In [113]:
# distances matrix
stacked_arr=np.vstack(distances)
distances_arr=np.transpose(stacked_arr)
distances_arr #-> 274 X 4

array([[0.44251985, 2.60805818, 1.56260145, 1.0773043 ],
       [0.44338015, 2.61646126, 1.54689978, 1.09447048],
       [0.36833168, 2.51987635, 1.45386457, 1.05341729],
       ...,
       [0.39945339, 2.12496156, 1.23894861, 0.76135433],
       [0.45422115, 2.02341986, 1.07570108, 0.88248018],
       [0.58633963, 2.02405096, 1.27847725, 0.77663493]])

In [114]:
radiis

[1.000000930168902, 1.0000007431025906, 0.999999537671207, 1.0001279549432047]

In [115]:
check_anamoly=radius_cluster-distances_arr     # positive = inside cluster | negative = outside cluster
check_anamoly

array([[ 0.55748108, -1.60990071, -0.56400135, -0.07717635],
       [ 0.55662078, -1.61830379, -0.54829968, -0.09434253],
       [ 0.63166925, -1.52171888, -0.45526447, -0.05328933],
       ...,
       [ 0.60054754, -1.12680409, -0.24034851,  0.23877362],
       [ 0.54577978, -1.0252624 , -0.07710098,  0.11764777],
       [ 0.4136613 , -1.0258935 , -0.27987714,  0.22349302]])

In [116]:
check_anamoly.shape

(826, 4)

In [124]:
predicted_attack=[266,281,285,287]
for i in predicted_attack:
    print(distances_arr[i])
actual_attack=[803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823]
print("actual attack distances")
for id in actual_attack:
    print(distances_arr[id])

[1.49273791 1.07260137 1.0116522  1.25145955]
[1.72315789 1.00600252 1.05368885 1.17854858]
[1.53328746 1.59572615 1.05469944 1.22402048]
[1.18079238 2.00360505 1.45839561 1.2257041 ]
actual attack distances
[1.5694596  1.70853139 1.50378283 0.64759405]
[0.43653115 2.6142627  1.52135289 1.11275276]
[0.40795299 2.60761556 1.57185303 1.05373144]
[0.45384667 2.64318132 1.55802663 1.11888689]
[0.44982428 2.63993156 1.56056807 1.11491571]
[0.46211785 2.68482256 1.61589227 1.1171387 ]
[0.48442441 2.70632986 1.60316078 1.14878102]
[0.42671658 1.96048962 1.01604891 0.84267024]
[0.34871469 1.863112   1.0722112  0.66710999]
[0.337346   1.92086961 1.03747553 0.74596909]
[0.39945339 2.12496156 1.23894861 0.76135433]


radius-[1.000000930168902, 0.998157468152326, 0.9986001021045486, 1.0001279549432047]

In [121]:
predicted_attack=[266,281,285,287]
actual_attack=[803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823]
print("predicted attack")
for id in predicted_attack:
    print(check_anamoly[id])
print("actual attack")
for id in actual_attack:
    print(check_anamoly[id])

predicted attack
[-0.49273698 -0.0744439  -0.0130521  -0.25133159]
[-0.72315696 -0.00784505 -0.05508874 -0.17842063]
[-0.53328653 -0.59756868 -0.05609934 -0.22389252]
[-0.18079145 -1.00544758 -0.45979551 -0.22557614]
actual attack
[-0.56945867 -0.71037392 -0.50518273  0.35253391]
[ 0.56346978 -1.61610524 -0.52275278 -0.1126248 ]
[ 0.59204794 -1.60945809 -0.57325293 -0.05360348]
[ 0.54615426 -1.64502386 -0.55942653 -0.11875894]
[ 0.55017665 -1.6417741  -0.56196796 -0.11478776]
[ 0.53788309 -1.68666509 -0.61729217 -0.11701074]
[ 0.51557652 -1.70817239 -0.60456067 -0.14865306]
[ 0.57328435 -0.96233215 -0.01744881  0.15745771]
[ 0.65128624 -0.86495453 -0.0736111   0.33301796]
[ 0.66265493 -0.92271214 -0.03887542  0.25415887]
[ 0.60054754 -1.12680409 -0.24034851  0.23877362]


In [118]:
#check for if any datapoint is coming inside the cluster
is_positive=np.any(check_anamoly>0,axis=1)
#indices of those rows
positive_indices=np.where(is_positive)[0]


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [119]:
for i in range(826):
  if i not in positive_indices:
    print(i)

266
281
285
287


[803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823])

In [120]:
len(positive_indices)

822